In [76]:
import os
import torch
import random
import torch.nn as nn
import torch.backends.cudnn as cudnn
from models import build_model
import numpy as np
from PIL import Image

In [77]:
a = torch.arange(10)
a[-3:]

tensor([7, 8, 9])

In [78]:
!nvidia-smi

Mon Nov 25 01:02:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:C0:00.0 Off |                    0 |
| N/A   25C    P0              65W / 400W |  43062MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [79]:
from huggingface_hub import snapshot_download

#model_path = '/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models'   # The local directory to save Video-LaVIT checkpoint
#snapshot_download("rain1011/Video-LaVIT-v1", local_dir=model_path, local_dir_use_symlinks=False, repo_type='model')


In [80]:
#shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/video_lavit_for_understanding.py

In [81]:
# The local directory to save LaVIT checkpoint, set to yours
model_path = "/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft"#"/home/jinyang06/models/VideoLaVIT-v1/language_model_sft"
model_dtype='bf16'

max_video_clips = 16
device_id = 0
torch.cuda.set_device(device_id)
device = torch.device('cuda')

seed = 42
#torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# For Multi-Modal Understanding
runner = build_model(model_path=model_path, model_dtype=model_dtype, understanding=True, 
        device_id=device_id, use_xformers=False, max_video_clips=max_video_clips,)
print("Building Model Finsished")

Loading Video LaVIT Model Weight from /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft, model precision: bf16
Not used {}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft were not used when initializing VideoLaVITLlamaForCausalLM: ['model.motion_tokenizer.quantize.embedding.cluster_size', 'model.motion_tokenizer.quantize.embedding.embed_avg', 'model.motion_tokenizer.quantize.cluster_size', 'model.motion_tokenizer.quantize.embedding.initted']
- This IS expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The Visual Vocab Size is 16384
The llama tokenizer vocab size is 32000
The maximal clip number is 16
Building Model Finsished


In [82]:
#input = ffmpeg.input('/home/chekalina/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/demo/video7013.mp4')
#video = input.video.filter("aecho", 0.8, 0.9, 1000, 0.3)
#video = input.video.hflip()
#out = ffmpeg.output(audio, video, 'out.mp4')

In [83]:
#!pip install --upgrade transformers

### Image Understanding

In [84]:
#import numpy as np
#image_path = 'demo/03-Confusing-Pictures.jpg'
#prompt = "What is unusual about this image?"

#output = runner({"image": image_path, "text_input": prompt}, length_penalty=1, temperature=1.0, \
#                use_nucleus_sampling=False, num_beams=1, truct_vqa=False, max_length=512,)[0]
#print(output)


In [85]:
#import numpy as np
#image_path = 'demo/03-Confusing-Pictures.jpg'
#prompt = "What is unusual about this image?"

#runner.get_logits({"image": image_path, "text_input": prompt}, num_beams=1, truct_vqa=False)

### Video Understanding

In [86]:
from models.transform import LaVITImageProcessor, LaVITEvalVideoProcessor
video_processor = LaVITEvalVideoProcessor(image_size=224, num_frames=24, fps=6, max_clips=8,)
visual_inputs, motion_inputs = video_processor('demo/video7013.mp4')

done!
done!
done!


In [87]:
print (visual_inputs.shape, motion_inputs.shape)

torch.Size([3, 3, 224, 224]) torch.Size([3, 2, 24, 20, 36])


In [88]:
a = runner.model.encode_visual_features([([visual_inputs, motion_inputs], 'video')])

In [89]:
a[0].shape

torch.Size([692, 4096])

In [90]:
len(a)

1

In [91]:
visual_inputs, motion_inputs = video_processor('/home/jovyan/shares/SR004.nfs2/data/LLaVA-Video-178K/LLaVA-Video-178K/0_30_s_academic_v0_1/academic_source/Charades/028CE.mp4')


done!
done!
done!


In [92]:
print (visual_inputs.shape, motion_inputs.shape)

torch.Size([4, 3, 224, 224]) torch.Size([4, 2, 24, 20, 36])


In [93]:
a = runner.model.encode_visual_features([([visual_inputs, motion_inputs], 'video')])
a[0].shape

torch.Size([923, 4096])

In [94]:
visual_inputs.shape

torch.Size([4, 3, 224, 224])

In [95]:
motion_inputs.shape

torch.Size([4, 2, 24, 20, 36])

In [96]:
#%pdoc runner.model.generate

In [97]:
#%pfile runner.model.generate

In [98]:
#%runner.model.forward??

In [99]:
video_path = '/home/jovyan/shares/SR004.nfs2/data/LLaVA-Video-178K/LLaVA-Video-178K/0_30_s_academic_v0_1/academic_source/Charades/028CE.mp4'
prompt = "Where does the video take place?"
answer = "In the kitchen"

#output = runner({"video": video_path, "text_input": prompt}, length_penalty=1, \
        #use_nucleus_sampling=False, num_beams=1, max_length=512, temperature=1.0)[0]
#print(output)

In [100]:
from conversation import default_conversation, DEFAULT_IMAGE_TOKEN, DEFAULT_VIDEO_TOKEN, VIDEO_TOKEN_INDEX, IMAGE_TOKEN_INDEX

DEFAULT_VISUAL_TOKEN = DEFAULT_VIDEO_TOKEN
VISUAL_TOKEN_INDEX = VIDEO_TOKEN_INDEX

In [101]:
import transformers
tokenizer = transformers.LlamaTokenizer.from_pretrained("/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft", use_fast=False, padding_side='left')
tokenizer.pad_token = tokenizer.unk_token

In [102]:
prompt = "Where does the video take place?"
text_inputs = [DEFAULT_VISUAL_TOKEN + "\n" + text_input for text_input in [prompt]]

In [103]:
from models.video_lavit_for_understanding import tokenizer_image_token

In [104]:
visual_inputs, motion_inputs = video_processor('/home/jovyan/shares/SR004.nfs2/data/LLaVA-Video-178K/LLaVA-Video-178K/0_30_s_academic_v0_1/academic_source/Charades/028CE.mp4')


done!
done!
done!


In [105]:
images = [([visual_inputs, motion_inputs], 'video')]

In [106]:
ele = images[0]
images[0][0][0].shape

torch.Size([4, 3, 224, 224])

In [107]:
for answer in answers:
    print (answer)

In the kitchen


In [108]:
from models.transform import LaVITImageProcessor, LaVITEvalVideoProcessor
video_processor = LaVITEvalVideoProcessor(image_size=224, num_frames=24, fps=6, max_clips=8,)
visual_inputs, motion_inputs = video_processor(video_path)

imgs = [([visual_inputs, motion_inputs], 'video')]
answers = ["In the kitchen"]

prompt = "Where does the video take place?"
text_inputs = [DEFAULT_VISUAL_TOKEN + "\n" + text_input for text_input in [prompt]]

prompts = []
for text_input in text_inputs:
    conv = default_conversation.copy()
    conv.append_message(conv.roles[0], text_input)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()
    prompts.append(prompt)

#print(prompts)

input_ids = [tokenizer_image_token(prompt, tokenizer, VISUAL_TOKEN_INDEX, DEFAULT_VISUAL_TOKEN, return_tensors='pt') for prompt in prompts]

#print("input_ids:", input_ids)

labels = [tokenizer(answer, return_tensors='pt').input_ids[0] for answer in answers]

ids_with_labels = [torch.cat((input_id, label), 0) for input_id, label in zip(input_ids, labels)]

len_labels = [len(label) for label in labels] 


#print (ids_with_labels)

#print("regular input_ids:", tokenizer(prompts).input_ids)
# To pad the token
ids_with_labels = torch.nn.utils.rnn.pad_sequence(
        ids_with_labels,
        batch_first=True,
        padding_value=tokenizer.pad_token_id,
).to(device)

#print("input_ids22:", ids_with_labels)

attention_mask = ids_with_labels.ne(tokenizer.pad_token_id)



done!
done!
done!


In [109]:
ids_with_labels

tensor([[    1,   319, 13563,  1546,   263, 12758,  1404,   322,   385, 23116,
         21082, 20255, 29889,   450, 20255,  4076,  8444, 29892, 13173, 29892,
           322,  1248,   568,  6089,   304,   278,  1404, 29915, 29879,  5155,
         29889,  3148,  1001, 29901, 29871,  -201, 29871,    13, 11921,   947,
           278,  4863,  2125,  2058, 29973,   319,  1799,  9047, 13566, 29901,
             1,   512,   278, 29181]], device='cuda:0')

In [110]:
ids_with_labels

tensor([[    1,   319, 13563,  1546,   263, 12758,  1404,   322,   385, 23116,
         21082, 20255, 29889,   450, 20255,  4076,  8444, 29892, 13173, 29892,
           322,  1248,   568,  6089,   304,   278,  1404, 29915, 29879,  5155,
         29889,  3148,  1001, 29901, 29871,  -201, 29871,    13, 11921,   947,
           278,  4863,  2125,  2058, 29973,   319,  1799,  9047, 13566, 29901,
             1,   512,   278, 29181]], device='cuda:0')

In [113]:
position_ids = torch.arange(0, ids_with_labels.shape[1], dtype=torch.long, device=ids_with_labels.device)

In [114]:
print ("images", len(imgs), len(imgs[0]), len(imgs[0][0]), imgs[0][0][0].shape, imgs[0][0][1].shape)

images 1 2 2 torch.Size([4, 3, 224, 224]) torch.Size([4, 2, 24, 20, 36])


In [116]:
ids_with_labels

tensor([[    1,   319, 13563,  1546,   263, 12758,  1404,   322,   385, 23116,
         21082, 20255, 29889,   450, 20255,  4076,  8444, 29892, 13173, 29892,
           322,  1248,   568,  6089,   304,   278,  1404, 29915, 29879,  5155,
         29889,  3148,  1001, 29901, 29871,  -201, 29871,    13, 11921,   947,
           278,  4863,  2125,  2058, 29973,   319,  1799,  9047, 13566, 29901,
             1,   512,   278, 29181]], device='cuda:0')

In [115]:
out = runner.model(input_ids = ids_with_labels, attention_mask = attention_mask, position_ids = position_ids, labels = None, 
        past_key_values = None,
        inputs_embeds = None, images = images)




before prepare_inputs_labels_for_multimodal
len(input_ids) 54
input_ids tensor([[    1,   319, 13563,  1546,   263, 12758,  1404,   322,   385, 23116,
         21082, 20255, 29889,   450, 20255,  4076,  8444, 29892, 13173, 29892,
           322,  1248,   568,  6089,   304,   278,  1404, 29915, 29879,  5155,
         29889,  3148,  1001, 29901, 29871,  -201, 29871,    13, 11921,   947,
           278,  4863,  2125,  2058, 29973,   319,  1799,  9047, 13566, 29901,
             1,   512,   278, 29181]], device='cuda:0')



input_ids.shape torch.Size([54])
labels.shape torch.Size([54])
num images! tensor(1, device='cuda:0')
image_token_indices! [-1, 35, 54]
new_input_embeds.shape torch.Size([968, 4096])
new_labels.shape torch.Size([968])
new_labels [tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
 

In [ ]:
a = torch.arange(10)
a[-3:]

In [ ]:
out.logits.shape

In [ ]:
use_nucleus_sampling=True
num_beams=1
max_length=512
min_length=8
top_p=1.0
top_k=50
repetition_penalty=1
length_penalty=1
num_return_sequences=1
temperature=1
truct_vqa=False
stop_str='\\n'
lemmatize=False

In [38]:
from utils import get_rank, KeywordsStoppingCriteria
supress_range = 32000 + runner.visual_vocab_size + runner.motion_vocab_size + 2
suppress_tokens = [x for x in range(32000, supress_range)]

keywords = [stop_str]
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

In [ ]:
output_ids = runner.model.generate(
                input_ids,
                images=images,
                attention_mask=attention_mask,
                do_sample=use_nucleus_sampling,
                temperature=temperature,
                num_beams=num_beams,
                min_new_tokens=min_length,
                max_new_tokens=max_length,
                top_p=top_p,
                use_cache=True,
                suppress_tokens=suppress_tokens,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                repetition_penalty=repetition_penalty,
                length_penalty=length_penalty,
                num_return_sequences=num_return_sequences,
                stopping_criteria=[stopping_criteria],
            )

In [34]:
output_ids.shape

torch.Size([1, 78])

In [40]:
print("input_ids.shape[1]:", input_ids.shape[1])

input_ids.shape[1]: 50


In [41]:
input_token_len = input_ids.shape[1]
n_diff_input_output = (input_ids != output_ids[:, :input_token_len]).sum().item()
if n_diff_input_output > 0:
    print(
        f"[Warning] {n_diff_input_output} output_ids are not the same as the input_ids"
        )
outputs = tokenizer.batch_decode(output_ids[:, input_token_len:], skip_special_tokens=True
)
outputs = [output.split('\\n')[0] for output in outputs]
outputs = [output.strip() for output in outputs]
outputs

['The video takes place in a workroom or garage and appears to be related to cleaning or household tasks as the woman is consistently wearing black clothes and is seen picking up various household cleaning products. The background color scheme is predominantly dark with red and black accents.']

## More presicely

In [15]:
from datasets import load_dataset
dataset = load_dataset("Sayankotor/video_dataset_qa")

README.md:   0%|          | 0.00/425 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/70.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/715110 [00:00<?, ? examples/s]

In [17]:
dataset['train'][0]

{'question': '<image>\nWhere does the video take place?',
 'answer': 'The video takes place in a garage or basement.',
 'video_path': '/home/jovyan/shares/SR004.nfs2/data/LLaVA-Video-178K/LLaVA-Video-178K/0_30_s_academic_v0_1/academic_source/Charades/028CE.mp4',
 'vlen': '0-30',
 'task': 'open_ended'}

In [16]:
video_path = '/home/jovyan/shares/SR004.nfs2/data/LLaVA-Video-178K/LLaVA-Video-178K/0_30_s_academic_v0_1/academic_source/Charades/028CE.mp4'
prompt = "Where does the video take place?"

output = runner({"video": video_path, "text_input": prompt}, length_penalty=1, \
        use_nucleus_sampling=False, num_beams=1, max_length=512, temperature=1.0)[0]
print(output)

done!
done!
done!
["A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: <video>\nWhere does the video take place? ASSISTANT:"]
input_ids: [tensor([    1,   319, 13563,  1546,   263, 12758,  1404,   322,   385, 23116,
        21082, 20255, 29889,   450, 20255,  4076,  8444, 29892, 13173, 29892,
          322,  1248,   568,  6089,   304,   278,  1404, 29915, 29879,  5155,
        29889,  3148,  1001, 29901, 29871,  -201, 29871,    13, 11921,   947,
          278,  4863,  2125,  2058, 29973,   319,  1799,  9047, 13566, 29901])]
len images 1
len(visual_input_tensor), visual_input_tensor.shape 4 torch.Size([3, 224, 224])
len(visual_embeds_list), visual_embeds_list.shape 4 torch.Size([88, 4096])
len(motion_input_tensor), motion_input_tensor.shape 4 torch.Size([2, 24, 20, 36])
len(motion_embeds), motion_embeds.shape 4 torch.Size([4, 137, 4096])
The video takes place in a garage, where 

In [23]:
from models.video_lavit_for_understanding import tokenizer_image_token

In [26]:
from models.transform import LaVITImageProcessor, LaVITEvalVideoProcessor
video_processor = LaVITEvalVideoProcessor(image_size=224, num_frames=24, fps=6, max_clips=8,)

In [18]:
from conversation import default_conversation, DEFAULT_IMAGE_TOKEN, DEFAULT_VIDEO_TOKEN, VIDEO_TOKEN_INDEX, IMAGE_TOKEN_INDEX
visual_inputs, motion_inputs = video_processor(video_path)
DEFAULT_VISUAL_TOKEN = DEFAULT_VIDEO_TOKEN
VISUAL_TOKEN_INDEX = VIDEO_TOKEN_INDEX

done!
done!
done!


In [19]:
prompt = "Where does the video take place?"
text_inputs = [DEFAULT_VISUAL_TOKEN + "\n" + text_input for text_input in [prompt]]

In [21]:
text_inputs

['<video>\nWhere does the video take place?']

In [30]:
default_conversation

Conversation(system="A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.", roles=('USER', 'ASSISTANT'), messages=(), offset=0, sep_style=<SeparatorStyle.TWO: 2>, sep=' ', sep2=' \\n </s>', version='v1', skip_next=False)

In [26]:
import transformers
tokenizer = transformers.LlamaTokenizer.from_pretrained("/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft", use_fast=False, padding_side='left')

In [31]:

prompts = []
for text_input in text_inputs:
    conv = default_conversation.copy()
    conv.append_message(conv.roles[0], text_input)
    conv.append_message(conv.roles[1], None)
    prompt = conv.get_prompt()
    prompts.append(prompt)

print(prompts)

input_ids = [tokenizer_image_token(prompt, tokenizer, VISUAL_TOKEN_INDEX, DEFAULT_VISUAL_TOKEN, return_tensors='pt') for prompt in prompts]

print("input_ids:", input_ids)
# To pad the token
input_ids = torch.nn.utils.rnn.pad_sequence(
        input_ids,
        batch_first=True,
        padding_value=tokenizer.pad_token_id,
).to(device)

attention_mask = input_ids.ne(tokenizer.pad_token_id)

["A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: <video>\nWhere does the video take place? ASSISTANT:"]
input_ids: [tensor([    1,   319, 13563,  1546,   263, 12758,  1404,   322,   385, 23116,
        21082, 20255, 29889,   450, 20255,  4076,  8444, 29892, 13173, 29892,
          322,  1248,   568,  6089,   304,   278,  1404, 29915, 29879,  5155,
        29889,  3148,  1001, 29901, 29871,  -201, 29871,    13, 11921,   947,
          278,  4863,  2125,  2058, 29973,   319,  1799,  9047, 13566, 29901])]


In [32]:
attention_mask

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True, True, True, True, True,
         True, True]], device='cuda:0')

In [ ]:
runner.model.prepare_inputs_labels_for_multimodal

runner.model prepare_inputs_labels_for_multimodal